# Customer Churn Analysis and Prediciton

The dataset is posted on [Kaggle](https://www.kaggle.com/datasets/blastchar/telco-customer-churn).


In [17]:
import pandas as pd
import numpy as np
import altair as alt
import statsmodels.formula.api as smf

alt.data_transformers.enable("vegafusion")

DataTransformerRegistry.enable('vegafusion')

In [25]:
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.metrics import (
    recall_score,
    confusion_matrix,
    precision_score,
    f1_score,
    accuracy_score,
    classification_report,
)
from xgboost import XGBClassifier

# Data Preparation

In [3]:
df = pd.read_csv("WA_Fn-UseC_-Telco-Customer-Churn.csv")
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


## Data Cleaning

- There are 11 samples with TotalCharges = 0. Note that their tenure are also 0. We guess they are new to the service and haven't been charged.
- Drop 11 samples with `TotalCharges == 0`

In [4]:
df.drop(columns=["customerID"], inplace=True)
df["TotalCharges"] = pd.to_numeric(df.TotalCharges, errors="coerce")
print(df.isnull().sum())
df.dropna(inplace=True)
print(df.isnull().sum())

gender               0
SeniorCitizen        0
Partner              0
Dependents           0
tenure               0
PhoneService         0
MultipleLines        0
InternetService      0
OnlineSecurity       0
OnlineBackup         0
DeviceProtection     0
TechSupport          0
StreamingTV          0
StreamingMovies      0
Contract             0
PaperlessBilling     0
PaymentMethod        0
MonthlyCharges       0
TotalCharges        11
Churn                0
dtype: int64
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64


# EDA

In [5]:
chart = (
    alt.Chart(df)
    .mark_bar(
        opacity=0.4,
        binSpacing=0,
    )
    .encode(
        alt.X("tenure:Q").bin(maxbins=50),
        alt.Y("count()").stack(None),
        alt.Color("Contract:N"),
    )
    .properties(
        title="Histogram of Tenure",
    )
)

# chart.save(f"imgs/Histogram_of_Tenure.png", ppi=200)
chart.display()

alt.Chart(...)

- Different contract types do show different distributions of tenure.
- The tenure of monthly contract is short; otherwise, it's relatively high in two-year contract.

In [11]:
chart = (
    alt.Chart(df)
    .mark_bar(
        opacity=0.4,
        binSpacing=0,
    )
    .encode(
        alt.X("MonthlyCharges:Q").bin(maxbins=50),
        alt.Y("count()").stack(None),
        alt.Color("Contract:N"),
    )
    .properties(
        title="Histogram of MonthlyCharges",
    )
)

# chart.save(f"imgs/Histogram_of_MonthlyCharges.png", ppi=200)
chart.display()

alt.Chart(...)

- Monthly charges seem to be non-correlated to contract type especially for the monthly charge < 30. They share similiar fees among contract types.
- There are other factors affecting the monthly fees. We conduct additional regression analysis to investigate the relationship.

## Price Analysis
- Conduct regression analysis on the monthly charges

In [12]:
# Price analysis

ols_df = df[
    [
        "MultipleLines",
        "InternetService",
        "OnlineSecurity",
        "OnlineBackup",
        "DeviceProtection",
        "TechSupport",
        "MonthlyCharges",
        "StreamingTV",
        "StreamingMovies",
    ]
]

# ols_df.loc[:, 'DeviceProtection'] = ols_df.DeviceProtection.str.replace("No internet service", "No")
# ols_df.loc[:, 'OnlineBackup'] = ols_df.DeviceProtection.str.replace("No internet service", "No")
# ols_df.loc[:, 'OnlineSecurity'] = ols_df.DeviceProtection.str.replace("No internet service", "No")
# ols_df.loc[:, 'TechSupport'] = ols_df.DeviceProtection.str.replace("No internet service", "No")

# ols_df.loc[:, 'StreamingMovies'] = ols_df.DeviceProtection.str.replace("No internet service", "No")
# ols_df.loc[:, 'StreamingTV'] = ols_df.DeviceProtection.str.replace("No internet service", "No")

# ols_df.loc[:, 'MultipleLines'] = ols_df.DeviceProtection.str.replace("No phone service", "No")

inde_x = ols_df.columns.difference(["MonthlyCharges"])
inde_x = list(map(lambda x: f"C({x})", inde_x))

model = smf.ols(formula=f'MonthlyCharges ~ {"+".join(inde_x)} ', data=ols_df)

results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:         MonthlyCharges   R-squared:                       0.999
Model:                            OLS   Adj. R-squared:                  0.999
Method:                 Least Squares   F-statistic:                 6.023e+05
Date:                Sat, 13 Jul 2024   Prob (F-statistic):               0.00
Time:                        21:01:01   Log-Likelihood:                -10162.
No. Observations:                7032   AIC:                         2.035e+04
Df Residuals:                    7021   BIC:                         2.042e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                                                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------

In [13]:
df_price = ols_df.drop(columns=["MonthlyCharges"]).drop_duplicates()
idx = df_price.index
df_price = df_price.reset_index(drop=True)
price_true = ols_df.loc[idx, "MonthlyCharges"].values
df_price["price_est"] = results.predict(exog=df_price.to_dict("list"))
df_price["price_%"] = np.abs(df_price["price_est"] - price_true) / price_true
print(f"MAPE: {np.round(np.mean(df_price['price_%']), 4)*100}%")
df_price.sort_values(by="price_est", ascending=False).round(4)

MAPE: 1.22%


,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,price_est,price_%
13,Yes,Fiber optic,Yes,Yes,Yes,Yes,Yes,Yes,114.9495,0.0150
50,Yes,Fiber optic,Yes,No,Yes,Yes,Yes,Yes,109.9556,0.0147
43,Yes,Fiber optic,No,Yes,Yes,Yes,Yes,Yes,109.9396,0.0137
233,No,Fiber optic,Yes,Yes,Yes,Yes,Yes,Yes,109.9346,0.0015
108,Yes,Fiber optic,Yes,Yes,No,Yes,Yes,Yes,109.9326,0.0179
...,...,...,...,...,...,...,...,...,...,...
7,No phone service,DSL,Yes,No,No,No,No,No,29.9491,0.0067
0,No phone service,DSL,No,Yes,No,No,No,No,29.9330,0.0028
44,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,25.1237,0.0070
67,No phone service,DSL,No,No,No,No,No,No,24.9392,0.0263


- The regression analysis (ANCOVA) expalins 99.9% variance on the means of monthly charges. ($R^2=0.999$)
- The model shows how Telco charges for each service. Therefore, we can list the charging fees corresponding to each combination of services.
- The MAPE of the estimatons is 1.22%.

## Churn Analysis

In [6]:
charts = {}
for col in [
    "gender",
    "SeniorCitizen",
    "Partner",
    "Dependents",
    "PhoneService",
    "MultipleLines",
    "InternetService",
    "OnlineSecurity",
    "OnlineBackup",
    "DeviceProtection",
    "TechSupport",
    "StreamingTV",
    "StreamingMovies",
    "Contract",
    "PaperlessBilling",
    "PaymentMethod",
]:
    chart = (
        alt.Chart(df)
        .mark_bar()
        .encode(
            x=alt.X(f"{col}:N").sort("-y"),
            y=alt.Y("count(Churn)").stack("normalize"),
            # column=f'{col}:N',
            color="Churn:N",
        )
        .properties(width=100, height=250)
    )
    charts[col] = chart

In [7]:
# Demographics
c = charts["SeniorCitizen"] | charts["Partner"] | charts["Dependents"]
c = c.properties(title="Demographics")
# c.save(f"imgs/demographics_churn.png", ppi=200)
c.display()

alt.HConcatChart(...)

- Senior citizen, people without partner and people without dependents are less likely to churn.

In [8]:
# Internet Service Related
c = (
    charts["InternetService"]
    | charts["OnlineSecurity"]
    | charts["OnlineBackup"]
    | charts["DeviceProtection"]
    | charts["TechSupport"]
    | charts["StreamingTV"]
    | charts["StreamingMovies"]
)
c = c.properties(title="Internet Service Related")
# c.save(f"imgs/internet_service_churn.png", ppi=200)
c.display()

alt.HConcatChart(...)

- For internet service, people choosing Fiber optic have the highest churnning rate.
- There are 4 additional service including online security, online backup, device protection and tech support sharing similar patterns. People adopting these service are less likey to churn (Yes vs. No), indicating their engagements.
- In addition, streaming TV and streaming movies are also related to internet service. But it seems these services may not satisfy customers a lot.

In [9]:
# Contract and Payment
c = charts["Contract"] | charts["PaperlessBilling"] | charts["PaymentMethod"]
c = c.properties(title="Contract and Payment")
# c.save(f"imgs/contract_payment_churn.png", ppi=200)
c.display()

alt.HConcatChart(...)

- The contract types do reflect user engagements. Month-to-month subscription users tend to leave Telco apparently.
- As for payment method, electronic check has the highest churn rate. People are more aware of how much they paid and perhaps think the service renewal twice.

## Baseline Model
We choose random forest model as the baseline model, since it's powerful enough and easy to understand.

In [10]:
categorical_cols = [
    "gender",
    "SeniorCitizen",
    "Partner",
    "Dependents",
    "PhoneService",
    "MultipleLines",
    "InternetService",
    "OnlineSecurity",
    "OnlineBackup",
    "DeviceProtection",
    "TechSupport",
    "StreamingTV",
    "StreamingMovies",
    "Contract",
    "PaperlessBilling",
    "PaymentMethod",
    "Churn",
]

df_baseline = pd.get_dummies(df, columns=categorical_cols, drop_first=True, dtype=int)
y = df_baseline["Churn_Yes"]
X = df_baseline.drop(columns=["Churn_Yes"])

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.10, random_state=42
)

In [11]:
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.82      0.91      0.86       511
           1       0.66      0.48      0.55       193

    accuracy                           0.79       704
   macro avg       0.74      0.69      0.71       704
weighted avg       0.78      0.79      0.78       704



- Random forest achives 79% accuracy and 48% recall wrt. churn.

# Feature Engineering

To measure user engagement, we can consider different dimensions: depth and breadth.
- Depth:
    * Which contract type the user chose?
    * How many times they have subscribed? (`tenure // month of contract`)
    * How many months are left until the next time renewal? (`tenure % month of contract`) 
- Breadth:
    * How many additional services they subscribed?
    * What's average price they'll pay for each service? (Cost Performance Index)

In [12]:
contract_month = {"Month-to-month": 1, "One year": 12, "Two year": 24}

# how many times they have subscribed
df["subscription_cycle"] = df["tenure"] // df.Contract.map(contract_month)

# how many months are left until the next time renewal
df["subscription_left"] = df["tenure"] % df.Contract.map(contract_month)
df["subscription_left_percent"] = (
    df["tenure"] % df.Contract.map(contract_month)
) / df.Contract.map(contract_month)


# how many additional services they subscribed
df["num_services"] = (
    df[
        [
            "PhoneService",
            "InternetService",
            "OnlineSecurity",
            "OnlineBackup",
            "DeviceProtection",
            "TechSupport",
            "StreamingTV",
            "StreamingMovies",
        ]
    ]
    == "Yes"
).sum(axis=1)

#  Cost Performance Index
df["cp"] = df["MonthlyCharges"] / (df["num_services"] + 1)

- Create dummy variables for categorical variables
- Split 10% dataset as test dataset
- Standardize numerical variables
- Create another feature: $tenure^2$, enhancing the longest and the shortest tenures

In [26]:
names = [
    "Nearest Neighbors",
    "RBF SVM",
    "Decision Tree",
    "Random Forest",
    "Neural Net",
    "AdaBoost",
    "Naive Bayes",
    "Logistic Regression",
    "XGBoost"
]

categorical_cols = [
    "gender",
    "SeniorCitizen",
    "Partner",
    "Dependents",
    "PhoneService",
    "MultipleLines",
    "InternetService",
    "OnlineSecurity",
    "OnlineBackup",
    "DeviceProtection",
    "TechSupport",
    "StreamingTV",
    "StreamingMovies",
    "Contract",
    "PaperlessBilling",
    "PaymentMethod",
    "Churn",
]

df_ml = pd.get_dummies(df, columns=categorical_cols, drop_first=True, dtype=int)
y = df_ml["Churn_Yes"]
X = df_ml.drop(columns=["Churn_Yes"])

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.10, random_state=42
)

numerical_cols = [
    "tenure",
    "MonthlyCharges",
    "TotalCharges",
    "subscription_cycle",
    "subscription_left",
    "subscription_left_percent",
    "num_services",
    "cp",
]
scaler = StandardScaler()
X_train[numerical_cols] = scaler.fit_transform(X_train[numerical_cols])
X_test[numerical_cols] = scaler.transform(X_test[numerical_cols])

X_train["tenure_sq"] = X_train["tenure"] ** 2
X_test["tenure_sq"] = X_test["tenure"] ** 2

## Feature Importance

In [14]:
forest = RandomForestClassifier(random_state=0)
forest.fit(X_train, y_train)
importances = forest.feature_importances_
std = np.std([tree.feature_importances_ for tree in forest.estimators_], axis=0)
forest_importances = pd.DataFrame(
    {"columns": X_train.columns, "importances": importances}
)

alt.Chart(forest_importances).mark_bar().encode(
    x=alt.X("importances:Q").title("Mean decrease in impurity"),
    y=alt.Y("columns:N").sort("-x"),
).properties(height=700, title="Feature importances using MDI")

alt.Chart(...)

- According to Feature Importance generated by random forest, the new features created earlier seem to be beneficial.

## Model Selection
- Use 5-fold cross-validation to evaluate different models.
- The performance are evaluated by accuracy and recall.

In [27]:
classifiers = [
    KNeighborsClassifier(5),
    SVC(kernel="rbf", gamma=2, C=1, random_state=42),
    DecisionTreeClassifier(random_state=42),
    RandomForestClassifier(random_state=42),
    MLPClassifier(alpha=1, max_iter=1000, random_state=42),
    AdaBoostClassifier(algorithm="SAMME", random_state=42),
    GaussianNB(),
    LogisticRegression(random_state=0, max_iter=100000, n_jobs=-1),
    XGBClassifier(random_state=42, n_jobs=-1)
]

result = []
for name, classifier in zip(names, classifiers):
    scores = cross_validate(classifier, X, y, scoring=["accuracy", "recall"])

    result.append(pd.DataFrame(scores).mean().values.tolist())

result_df = pd.DataFrame(result)
result_df.columns = ["fit_time", "score_time", "accuracy", "recall"]
result_df.index = names
result_df = result_df.sort_values(by=["recall", "accuracy"], ascending=False).round(2)
result_df

,fit_time,score_time,accuracy,recall
Naive Bayes,0.00,0.00,0.68,0.86
Neural Net,0.21,0.00,0.77,0.58
Logistic Regression,2.56,0.00,0.80,0.55
AdaBoost,0.18,0.01,0.80,0.52
XGBoost,0.13,0.01,0.78,0.51
Decision Tree,0.03,0.00,0.73,0.49
Random Forest,0.37,0.02,0.79,0.49
Nearest Neighbors,0.00,0.09,0.77,0.49
RBF SVM,0.75,0.53,0.74,0.04


- For customer churn, **recall** may be an important metric, since we want to increase the retention rate and take care of people who are likely to churn.
- Since recall and precision have a trade-off relationship, the precision is somewhat low. This indicates that the model is more conservative.
- Therefore, we choose **Naive Bayes** model to conduct predictions. 86% recall is relatively high compared to other models.

# Final Prediction
- Use the whole dataset to train the model

In [16]:
clf = GaussianNB()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.92      0.61      0.74       511
           1       0.46      0.86      0.60       193

    accuracy                           0.68       704
   macro avg       0.69      0.74      0.67       704
weighted avg       0.79      0.68      0.70       704



- The model's performance on the test dataset, with an 86% recall, is similar to that on the CV dataset.
- Telco should focus on customers with a high probability of leaving. They can provide promotions or coupons, or directly keep track of their activities.
- Identifying potential churn customers may increase the retention rate, consequently enhancing the corresponding customer lifetime value.